<a href="https://colab.research.google.com/github/Steve-YJ/deep-learning-from-scratch-studying/blob/master/Exp03_VAE_to_Malimg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# copy from Exp02. run_GAN

# drive mount
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

FOLDERNAME = '/Malimg_Exp_200611'

assert FOLDERNAME is not None, "[!] Enter the foldername."

%cd drive/My\ Drive
# %cp -r $FOLDERNAME ../../
# %cd ../../
# %cd /Malimg_Exp_200611/
# !bash get_datasets.sh
# %cd ../../

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/My Drive


## pickle 데이터 불러오기

In [2]:
import pickle

with open('/content/drive/My Drive/Malimg_Exp_200611/malimg_data2.pkl', 'rb') as f:
    data = pickle.load(f)

In [3]:
! pwd

/content/drive/My Drive


In [4]:
%cd Malimg_Exp_200611

/content/drive/My Drive/Malimg_Exp_200611


In [5]:
! pwd

/content/drive/My Drive/Malimg_Exp_200611


In [6]:
! pip install -r requirements.txt

In [7]:
# ! python main.py

# 잘 동작합니다

1st. 코드 파악하기<br>
* MNIST 데이터로 짜여진 VAE main.py 파악하기
* Encoder, Decoder, Loss Function 등 파악

* Again, Review를 잘 한것 같았으나... 코드가 동작하지 않음!
    * 아마 Custom DataLoader에 문제가 있지 않나 생각합니다...

## #1. Data Preprocessing

* Do-it
    * Data 구조 파악하기
    * Data Loader 만들기

### preprocessing1. numpy to tensor 

In [17]:
data[0].shape, data[1].shape

((9339, 50176), (9339,))

In [22]:
# type(data[0][0])
# make numpy array to tensor

In [23]:
import torch

# convert numpy to tensor
train_X = torch.from_numpy(data[0])
train_X.shape

torch.Size([9339, 50176])

In [24]:
train_X = train_X.reshape(-1, 1, 224, 224)
train_X.shape

torch.Size([9339, 1, 224, 224])

In [51]:
# train_X.transfrom()

AttributeError: ignored

In [25]:
# also convert y to tensor

train_y = torch.from_numpy(data[1])
train_y

tensor([ 0.,  0.,  0.,  ..., 24., 24., 24.], dtype=torch.float64)

In [27]:
train_y.shape, train_y.unique()

(torch.Size([9339]),
 tensor([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13.,
         14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24.],
        dtype=torch.float64))

In [28]:
train_X.shape, train_y.shape

(torch.Size([9339, 1, 224, 224]), torch.Size([9339]))

### Train_Test_Split이 필요하지 않나요?!

* Reconstruction을 하기 위해서는 test데이터가 필요
* <a href='https://github.com/pytorch/examples/blob/master/vae/main.py'>참고한 git</a>에서는 train, test data 사용

In [30]:
# train_test split
# Reference: https://github.com/Steve-YJ/deep-learning-from-scratch-studying/blob/master/02_PyTorch_Introduction_Basic/Ch05_5.5_%EC%99%80%EC%9D%B8%20%EB%B6%84%EB%A5%98%ED%95%98%EA%B8%B0.ipynb

from sklearn.model_selection import train_test_split

train_X, test_X, train_Y, test_Y = train_test_split(train_X, train_y, test_size=0.2)

# 데이터 확인
print(train_X.shape)
print(test_X.shape)

torch.Size([7471, 1, 224, 224])
torch.Size([1868, 1, 224, 224])


In [49]:
# train_X

### preprocessing 2. Make DataLoader

* train, test 모두 dataloader를 만들어줘야 한다

In [31]:
import torch
import numpy as np
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


* Reference: 
    * Reference:
PyTorch로 시작하는 딥러닝 입문: https://wikidocs.net/57165
    * Dataset 클래스 정의
    * Dataloader 만들기

In [55]:
class Malimg_train(torch.utils.data.Dataset):
    """Malimg dataset"""

    # Initialize your data, 
    def __init__(self):
        self.X_data = train_X
        self.y_data = train_Y

    def __getitem__(self, index):  ## torch.FlaotTensor()
                                   #  https://wikidocs.net/57165
        return torch.FloatTensor(self.X_data[index]), torch.FloatTensor(self.y_data[index])
    
    def __len__(self):
        return len(self.X_data)

In [56]:
class Malimg_test(torch.utils.data.Dataset):
    """Malimg dataset"""

    # Initialize your data, 
    def __init__(self):
        self.X_data = test_X
        self.y_data = test_Y

    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
    
    def __len__(self):
        return len(self.X_data)

In [57]:
## 클래스 인스턴스화
## Reference: https://tutorials.pytorch.kr/beginner/data_loading_tutorial.html

malimg_train = Malimg_train()
malimg_test = Malimg_test()

In [58]:
train_loader = DataLoader(malimg_train)
test_loader = DataLoader(malimg_test)

In [59]:
type(train_loader)

torch.utils.data.dataloader.DataLoader

## #2. VAE 코드 작성하기

* Reference: https://github.com/pytorch/examples/blob/master/vae/main.py

In [60]:
from __future__ import print_function
import argparse
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image


# parser = argparse.ArgumentParser(description='VAE MNIST Example')
# parser.add_argument('--batch-size', type=int, default=32, metavar='N',
#                     help='input batch size for training (default: 128)')
# parser.add_argument('--epochs', type=int, default=20, metavar='N',
#                     help='number of epochs to train (default: 10)')
# parser.add_argument('--no-cuda', action='store_true', default=False,
#                     help='enables CUDA training')
# parser.add_argument('--seed', type=int, default=1, metavar='S',
#                     help='random seed (default: 1)')
# parser.add_argument('--log-interval', type=int, default=10, metavar='N',
#                     help='how many batches to wait before logging training status')
# args = parser.parse_args()
# args.cuda = not args.no_cuda and torch.cuda.is_available()

# torch.manual_seed(args.seed)

device = torch.device("cuda")

# kwargs = {'num_workers': 1, 'pin_memory': True} if args.cuda else {}
# train_loader = torch.utils.data.DataLoader(
#     datasets.MNIST('../data', train=True, download=True,
#                    transform=transforms.ToTensor()),
#     batch_size=args.batch_size, shuffle=True, **kwargs)
# test_loader = torch.utils.data.DataLoader(
#     datasets.MNIST('../data', train=False, transform=transforms.ToTensor()),
#     batch_size=args.batch_size, shuffle=True, **kwargs)


class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()

        self.fc1 = nn.Linear(50176, 400)
        self.fc21 = nn.Linear(400, 20)
        self.fc22 = nn.Linear(400, 20)
        self.fc3 = nn.Linear(20, 400)
        self.fc4 = nn.Linear(400, 50176)

    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps*std

    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        return torch.sigmoid(self.fc4(h3))

    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, 50176))
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar


model = VAE().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)


# Reconstruction + KL divergence losses summed over all elements and batch
def loss_function(recon_x, x, mu, logvar):
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 50176), reduction='sum')

    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

    return BCE + KLD


def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.item() / len(data)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(train_loader.dataset)))


def test(epoch):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for i, (data, _) in enumerate(test_loader):
            data = data.to(device)
            recon_batch, mu, logvar = model(data)
            test_loss += loss_function(recon_batch, data, mu, logvar).item()
            if i == 0:
                n = min(data.size(0), 8)
                comparison = torch.cat([data[:n],
                                      recon_batch.view(32, 1, 224, 224)[:n]])
                save_image(comparison.cpu(),
                         'results/reconstruction_' + str(epoch) + '.png', nrow=n)

    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

if __name__ == "__main__":
    for epoch in range(1, 20 + 1):
        train(epoch)
        test(epoch)

        with torch.no_grad():
            sample = torch.randn(64, 20).to(device)
            sample = model.decode(sample).cpu()
            save_image(sample.view(64, 1, 224, 224),
                       'results/sample_' + str(epoch) + '.png')

TypeError: ignored